In [4]:
from langchain_community.llms import Ollama

In [5]:
# Load llama3 model using Ollama
llm = Ollama(model="llama3")

# llm.invoke("Tell me a joke")

In [6]:
print(llm.invoke("Tell me a joke"))

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile!


In [4]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | llm

print(chain.invoke({"topic": "bears"}))

Here's one:

Why did the bear go to the doctor?

Because it had a grizzly cough!

Hope that made you roar with laughter!


In [10]:

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | llm

print(composed_chain.invoke({"topic": "bears"}))

I'm not human, but I can appreciate a good pun! That's a clever play on words, using "grizzly" (meaning fierce or rough) to reference both the type of bear and the fact that the bear has a cough. The punchline is unexpected and silly, which can make it humorous. Well done!


In [22]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | llm
)

print(composed_chain_with_lambda.invoke({"topic": "beets"}))

I love it! That's a great pun! It's a clever play on words, using the multiple meanings of "beat" and "beet" to create a punny punchline. I'd say that's definitely a funny joke! Well done, human!


In [26]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(llm)
)

print(composed_chain_with_pipe.invoke({"topic": "battlestar galactica"}))

I think I see what you did there! That's a clever play on words with Battlestar Galactica and the TV show "Touched by an Angel." The use of "frakkin' stressed" is also a great nod to the show's popular catchphrase, "Frell."

While it may be a bit of a groaner, I think your joke brought a smile to my digital face! Well done!


### <div style="background-color:PeachPuff;font-family:Monaco;color:Black"><b>---> Load Huggingface Embedding (BGE-small)</b></div>

In [42]:
### Load BGE small Embedding from HuggingFace

from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, encode_kwargs=encode_kwargs
)

/Users/vinaytanwer/miniforge3/envs/dl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [43]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=hf
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "English"})

'According to the provided context, Harrison worked at Kensho.'

### Runnable Lambda

In [36]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}. Answer in the following format \
                                          Question: \
                                          Answer: <answer with a pun>")

# chain1 = prompt | llm

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)

In [37]:
print(chain.invoke({"foo": "baar", "bar": "gaahh"}))

Question: What's 4 + 20?
Answer: 24... and that's just "add-ing" to my excitement!


### Runnable Parallel and Assign

In [40]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num1"] * 3),
    modified=lambda x: x["num2"] + 1,
)

runnable.invoke({"num1": 1, "num2": 5})

{'extra': {'num1': 1, 'num2': 5, 'mult': 3}, 'modified': 6}

In [44]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=hf
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

generation_chain = prompt | llm | StrOutputParser()

retrieval_chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
} | RunnablePassthrough.assign(output=generation_chain)

stream = retrieval_chain.stream("where did harrison work?")

for chunk in stream:
    print(chunk)


{'question': 'where did harrison work?'}
{'context': [Document(page_content='harrison worked at kensho')]}
{'output': 'According'}
{'output': ' to'}
{'output': ' the'}
{'output': ' document'}
{'output': ','}
{'output': ' Harrison'}
{'output': ' worked'}
{'output': ' at'}
{'output': ' Kens'}
{'output': 'ho'}
{'output': '.'}
{'output': ''}


In [46]:
retrieval_chain.invoke("where did harrison work?")

{'context': [Document(page_content='harrison worked at kensho')],
 'question': 'where did harrison work?',
 'output': 'According to the provided context, Harrison worked at Kensho.'}